In [1]:
import pandas as pd
import os

In [2]:
import sys
def in_notebook():
    return 'ipykernel' in sys.modules

In [62]:
if not in_notebook():
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument("-g", "--gene_ann", help="Genes annotation", required=True)
    parser.add_argument("-c", "--cdna", help="Genes annotation", required=True)
    parser.add_argument("-d", "--deg", help="DEG genes", required=True)
    parser.add_argument("-s", "--ss", help="Shorstack results", required=True)
    parser.add_argument("--deseq", help="DEG genes", required=True)
    args = parser.parse_args()
    file_cdna = args.cdna
    file_deseq = args.deseq
    file_deg = args.deg
    file_results = args.ss
else:
    file_cdna = '/Volumes/toshiba/bio/TGAC/Triticum_aestivum.TGACv1.cdna.all.fa'
    file_results = 'data/21dpi_tgac/Results.txt'
    file_mart = 'data/mart_export.txt'
    file_deg = 'data/DEGs_by_RNAseq_21dpi_wheat.txt'
    file_deseq = 'data/files/diffexpr-results.0.1.csv'

In [4]:
#load DE genes
df_degenes = pd.read_csv(file_deg, delimiter='\t')
print('DEs genes:',len(df_degenes.index))

DEs genes: 3233


In [6]:
#load and extract sequences
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
fasta_seq = SeqIO.parse(file_cdna, 'fasta')
buff = []
gene_list = df_degenes.Gene.tolist()
dones = []
for record in fasta_seq:
    gene_name = record.id[:record.id.find('.')]
    if not gene_name in gene_list or gene_name in dones:
        dones.append(gene_name)
        continue
    buff.append(record)
SeqIO.write(buff, 'data/files/DEseqs.fa', "fasta")

5724

In [5]:
#search DE clusters
df_deseq = pd.read_csv(file_deseq,sep=',')

#load miRNA results file
df_mirna = pd.read_csv(file_results, delimiter='\t')
df_mirna = df_mirna[df_mirna.DicerCall != 'N']
print('miRNAs clusters:', len(df_mirna.index))

#DE miRNA
df_de_clusters = df_mirna[df_mirna.Name.isin(df_deseq.Gene)]
print('DE miRNA clusters:', len(df_de_clusters.index))

miRNAs clusters: 28845
DE miRNA clusters: 46


In [25]:

from subprocess import Popen, PIPE, call

#search targets
out_file = open("data/files/targets.txt","w") 
out_total = ''
for k,v in df_de_clusters.iterrows():
    cmd_list = ['./scripts/TargetFinder/targetfinder.pl','-s',v.MajorRNA,'-q',v.Name,'-d','data/files/DEseqs.fa','-p','gff']
    print(' '.join(cmd_list))
    pro = Popen(cmd_list, stdout=PIPE, stderr=PIPE)
    out,err = pro.communicate()
#    print(out,err)
    out_total += out.decode("utf-8") 
out_file.write(out_total)
out_file.close()
'''
os.system('grep -v "No results for" targets.txt > targets.clean.csv')
'''

./scripts/TargetFinder/targetfinder.pl -s UCCAUUUGUUGUUUUGAUUUC -q Cluster_1056 -d data/files/DEseqs.fa -p gff
b'TRIAE_CS42_5BS_TGACv1_423326_AA1374380.1 cdna scaffold:TGACv1:TGACv1_scaffold_423326_5BS:51756\ttargetfinder\trna_target\t60\t80\t4\t+\t.\tsmallRNA=Cluster_1056;target_seq=GAGAGAAAAACAAUAAAUGGG;base_pairs=::.:  :::::::.::::::.;miR_seq=CUUUAGUUUUGUUGUUUACCU\n' b''
./scripts/TargetFinder/targetfinder.pl -s UCCGAGGAUACUGAUGGACAG -q Cluster_1119 -d data/files/DEseqs.fa -p gff
b'No results for Cluster_1119\n' b''
./scripts/TargetFinder/targetfinder.pl -s AUAGAAGAAAGAACACAAAAA -q Cluster_2573 -d data/files/DEseqs.fa -p gff
b'TRIAE_CS42_6BS_TGACv1_515176_AA1667880.3 cdna scaffold:TGACv1:TGACv1_scaffold_515176_6BS:6256:\ttargetfinder\trna_target\t2178\t2198\t4\t+\t.\tsmallRNA=Cluster_2573;target_seq=UUUGUUUUUUCUUUCUUCUGU;base_pairs=::: : : :::::::::::.:;miR_seq=AAAAACACAAGAAAGAAGAUA\nTRIAE_CS42_6BS_TGACv1_515176_AA1667880.2 cdna scaffold:TGACv1:TGACv1_scaffold_515176_6BS:6256:\ttarg

b'No results for Cluster_15352\n' b''
./scripts/TargetFinder/targetfinder.pl -s AUCGAACAAUGGCAAAAAGAA -q Cluster_17414 -d data/files/DEseqs.fa -p gff
b'No results for Cluster_17414\n' b''
./scripts/TargetFinder/targetfinder.pl -s UAGAAAUACAUUAAUUGGAGA -q Cluster_18328 -d data/files/DEseqs.fa -p gff
b'TRIAE_CS42_5BL_TGACv1_407358_AA1356170.2 cdna scaffold:TGACv1:TGACv1_scaffold_407358_5BL:21673\ttargetfinder\trna_target\t1501\t1522\t4\t+\t.\tsmallRNA=Cluster_18328;target_seq=UCGUUUAAUUACUGUAUUUCUA;base_pairs=:: :..::::: ::::::::::;miR_seq=AG-AGGUUAAUUACAUAAAGAU\nTRIAE_CS42_5BL_TGACv1_407358_AA1356170.1 cdna scaffold:TGACv1:TGACv1_scaffold_407358_5BL:21673\ttargetfinder\trna_target\t1495\t1516\t4\t+\t.\tsmallRNA=Cluster_18328;target_seq=UCGUUUAAUUACUGUAUUUCUA;base_pairs=:: :..::::: ::::::::::;miR_seq=AG-AGGUUAAUUACAUAAAGAU\nTRIAE_CS42_5BL_TGACv1_407358_AA1356170.3 cdna scaffold:TGACv1:TGACv1_scaffold_407358_5BL:21673\ttargetfinder\trna_target\t1458\t1479\t4\t+\t.\tsmallRNA=Cluster_18328;

b'No results for Cluster_34883\n' b''
./scripts/TargetFinder/targetfinder.pl -s UUUGAACAGAGUAGUGGCAUC -q Cluster_36390 -d data/files/DEseqs.fa -p gff
b'No results for Cluster_36390\n' b''
./scripts/TargetFinder/targetfinder.pl -s UGACAAAAGAACGACGGUAAGU -q Cluster_36391 -d data/files/DEseqs.fa -p gff
b'No results for Cluster_36391\n' b''
./scripts/TargetFinder/targetfinder.pl -s AGCAGCGAAUGGCACGAACUG -q Cluster_36392 -d data/files/DEseqs.fa -p gff
b'TRIAE_CS42_3AL_TGACv1_194242_AA0629460.1 cdna scaffold:TGACv1:TGACv1_scaffold_194242_3AL:72281\ttargetfinder\trna_target\t1040\t1060\t3.5\t+\t.\tsmallRNA=Cluster_36392;target_seq=GGGUUCUCGCCAUUCGCUGCU;base_pairs= .::::  :::::::::::::;miR_seq=GUCAAGCACGGUAAGCGACGA\nTRIAE_CS42_3B_TGACv1_223369_AA0781120.1 cdna scaffold:TGACv1:TGACv1_scaffold_223369_3B:53817:5\ttargetfinder\trna_target\t875\t895\t3.5\t+\t.\tsmallRNA=Cluster_36392;target_seq=GGGUUCUCGCCAUUCGCUGCU;base_pairs= .::::  :::::::::::::;miR_seq=GUCAAGCACGGUAAGCGACGA\n' b''
./scripts/Tar

'\nos.system(\'grep -v "No results for" targets.txt > targets.clean.csv\')\n'

In [36]:
df_targets = pd.read_csv('data/files/targets.txt', sep='\t', header=None)
df_targets['Gene'] = df_targets[0].str.split(' ').apply(lambda x: x[0].split('.')[0])
df_targets['cluster'] = df_targets[8].str.split(';').apply(lambda x: x[0].split('=')[1])

In [42]:
df_degenes.head(3)

,Gene,log2FC
0,TRIAE_CS42_2BL_TGACv1_130477_AA0412140,-2.456314
1,TRIAE_CS42_7AS_TGACv1_570678_AA1839270,4.295336
2,TRIAE_CS42_2BL_TGACv1_129652_AA0391760,2.744696


In [43]:
df_targets_2 = pd.merge(df_targets, df_degenes, on='Gene')
print('Upregulated',len(df_targets_2[df_targets_2.log2FC > 0].index))
print('Downregulated',len(df_targets_2[df_targets_2.log2FC < 0].index))
#df_de_clusters['Cluster'] = df_de_clusters['Name']
#df_targets_2 = pd.merge(df_targets, df_de_clusters[['Cluster','genes']], on='Cluster')
#df_targets_3 = df_targets_2[['Cluster','Gene','log2FC','genes']]
#df_targets_3.columns = ['miRNA Cluster','Target Gene','Target log2FC','Origin gene']
#df_targets_3.to_csv('data/targets_fc.csv',header=False, index=False, sep='\t')
#df_targets_3

Upregulated 49
Downregulated 20


In [71]:
#df_targets_2.Gene.unique().tolist()
df_targets_3 = pd.merge(df_targets_2, df_mirna, left_on=['cluster'], right_on=['Name'])
df_targets_3 = df_targets_3[df_targets_3.MIRNA == 'Y']

In [85]:
df_mart = pd.read_csv(file_mart, delimiter=',', quotechar = '"',escapechar='\\')
df_targets_mart = pd.merge(df_targets_2, df_mart, left_on=['Gene'], right_on=['Gene stable ID'])
df_targets_mart.to_csv('data/files/targets_mart.csv',sep='\t',index=None)
#gene_list = df_targets_2.Gene.unique().tolist()
#df_mart.head(1)
#for g in gene_list[0:4]:
#    df = df_mart[df_mart['Gene stable ID'] == g]
#    print(g)
#    print(df[['GO term definition', 'GO term name','GO domain']])


/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [78]:
df_mart.head(1)

,Gene stable ID,Transcript stable ID,Gene name,GO term accession,GO term name,GO term definition,GO domain,GO term evidence code
0,TRIAE_CS42_U_TGACv1_641895_AA2106830,TRIAE_CS42_U_TGACv1_641895_AA2106830.1,NaN,GO:0048316,seed development,"""The process whose specific outcome is the pro...",biological_process,IEA


In [ ]:
df_de_clusters['cluster'] = df_de_clusters['Name']
df_de_clusters_targets = pd.merge(df_targets[['target_gene','cluster']],df_de_clusters,on='cluster')
df_de_clusters_targets